In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [4]:
pd.pandas.set_option('display.max_rows', 50)
pd.pandas.set_option('display.max_columns', 1000)

In [5]:
rdf = pd.read_csv('Zomato Restaurants DataFrame - Clean & FeatEng II.csv', index_col=False)

In [6]:
rdf.head()

,Restaurant Name,Restaurant Area,Restaurant Rating,Restaurant Review,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info,Province,City,District/Kecamatan,Delivery Service,Place Type,Cuisine Regionality,Cuisines,acceptscryptocurrency,alkoholtersedia,areamerokok,bawapulangtersedia,biayabukabotol,biayamasuk,boardgames,brunch,buffet,byob,craftbeer,dessertsandbakes,didalamruangan,drivethrough,gastropub,hanyabawapulang,hanyabir,hanyapesanantar,hanyauntukreservasimeja,hanyawine,hanyawinedanbir,kaki5bersertifikatbpom,karaoke,kehidupanmalam,lantaidansa,liveentertainment,livemusic,makandengangrup,makanmewah,mejabilliard,menumakansiang,menyediakancocktail,menyediakanhalal,noadditionalinfo,parkirgratis,parkirvalettersedia,pemandangankota,pilihanbebasgluten,pilihanvegan,prasmanan,ramahuntukanak,ramahuntukdifabel,ramahvegetarian,reservasimejadirekomendasikan,rooftop,ruangpribaditersedia,ruangsholat,sarapan,shisha,sportsbar,tempatdudukdiluar,wifitersedia
0,Holy Smokes,Senopati,4.4,1437,-6.239143,106.807954,https://www.zomato.com/id/jakarta/holy-smokes-...,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",500000,"['Di dalam ruangan', 'Hanya Wine dan Bir', 'Pa...",DKI Jakarta,Jakarta Selatan,Kebayoran Baru,No,['Casual Dining'],['Barat'],"['Grill House', 'Steak']",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,Pempek Megaria,Cikini,4.3,461,-6.199900,106.843739,https://www.zomato.com/id/jakarta/pempek-megar...,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",70000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],['Palembang'],['Pempek'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tjikinii Lima,Cikini,4.0,391,-6.189802,106.837150,https://www.zomato.com/id/jakarta/tjikinii-lim...,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",300000,"['Ruang Pribadi Tersedia', 'Tempat duduk di lu...",DKI Jakarta,Jakarta Pusat,Menteng,No,['Casual Dining'],"['Indonesia', 'Barat']",['Kopi'],0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1
3,Gado-Gado Bon-Bin,Cikini,4.0,177,-6.192959,106.838377,https://www.zomato.com/id/jakarta/gado-gado-bo...,"Jl. Cikini 4 No. 5, Cikini, Jakarta",90000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Makanan Sehat'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,RM Wong Tegal,Tanjung Duren,3.9,12,-6.182315,106.782992,https://www.zomato.com/id/jakarta/rm-wong-tega...,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",100000,"['Bawa Pulang Tersedia', 'Sarapan', 'Di dalam ...",DKI Jakarta,Jakarta Barat,Grogol Petamburan,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Satay'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [7]:
# def convert_list(row, name): 
#     stripped = row[name].strip("[]")
#     splitted = list(stripped.split(", "))
#     final = [letter.strip("'") for letter in splitted]
#     return final

In [8]:
# rgdf['Restaurant Type'] = rgdf.apply(lambda row: convert_list(row, 'Restaurant Type'), axis=1)
# rgdf['Cuisine Type'] = rgdf.apply(lambda row: convert_list(row, 'Cuisine Type'), axis=1)
# rgdf['Cuisine Region Type'] = rgdf.apply(lambda row: convert_list(row, 'Cuisine Region Type'), axis=1)

# RECOMMENDER SYSTEM

In [9]:
rdf.head()

,Restaurant Name,Restaurant Area,Restaurant Rating,Restaurant Review,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info,Province,City,District/Kecamatan,Delivery Service,Place Type,Cuisine Regionality,Cuisines,acceptscryptocurrency,alkoholtersedia,areamerokok,bawapulangtersedia,biayabukabotol,biayamasuk,boardgames,brunch,buffet,byob,craftbeer,dessertsandbakes,didalamruangan,drivethrough,gastropub,hanyabawapulang,hanyabir,hanyapesanantar,hanyauntukreservasimeja,hanyawine,hanyawinedanbir,kaki5bersertifikatbpom,karaoke,kehidupanmalam,lantaidansa,liveentertainment,livemusic,makandengangrup,makanmewah,mejabilliard,menumakansiang,menyediakancocktail,menyediakanhalal,noadditionalinfo,parkirgratis,parkirvalettersedia,pemandangankota,pilihanbebasgluten,pilihanvegan,prasmanan,ramahuntukanak,ramahuntukdifabel,ramahvegetarian,reservasimejadirekomendasikan,rooftop,ruangpribaditersedia,ruangsholat,sarapan,shisha,sportsbar,tempatdudukdiluar,wifitersedia
0,Holy Smokes,Senopati,4.4,1437,-6.239143,106.807954,https://www.zomato.com/id/jakarta/holy-smokes-...,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",500000,"['Di dalam ruangan', 'Hanya Wine dan Bir', 'Pa...",DKI Jakarta,Jakarta Selatan,Kebayoran Baru,No,['Casual Dining'],['Barat'],"['Grill House', 'Steak']",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
1,Pempek Megaria,Cikini,4.3,461,-6.199900,106.843739,https://www.zomato.com/id/jakarta/pempek-megar...,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",70000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],['Palembang'],['Pempek'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tjikinii Lima,Cikini,4.0,391,-6.189802,106.837150,https://www.zomato.com/id/jakarta/tjikinii-lim...,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",300000,"['Ruang Pribadi Tersedia', 'Tempat duduk di lu...",DKI Jakarta,Jakarta Pusat,Menteng,No,['Casual Dining'],"['Indonesia', 'Barat']",['Kopi'],0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1
3,Gado-Gado Bon-Bin,Cikini,4.0,177,-6.192959,106.838377,https://www.zomato.com/id/jakarta/gado-gado-bo...,"Jl. Cikini 4 No. 5, Cikini, Jakarta",90000,"['Bawa Pulang Tersedia', 'Di dalam ruangan']",DKI Jakarta,Jakarta Pusat,Menteng,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Makanan Sehat'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,RM Wong Tegal,Tanjung Duren,3.9,12,-6.182315,106.782992,https://www.zomato.com/id/jakarta/rm-wong-tega...,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",100000,"['Bawa Pulang Tersedia', 'Sarapan', 'Di dalam ...",DKI Jakarta,Jakarta Barat,Grogol Petamburan,No,['Quick Bites'],"['Jawa', 'Indonesia']",['Satay'],0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [10]:
def minimize_str(row, name): 
    lst = row[name]
    lst = lst.strip("[] '")
    lst = lst.replace("'",'')
    lst = lst.replace(" ",'')
    return lst

# 1.1 RECOMMENDER - CUISINES

In [11]:
#Create Dataframe Content Based by Cuisine
rdf_csne = rdf[['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]
rdf_csne['Cuisines STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Cuisines'), axis=1)

In [12]:
rdf_csne.head(10)

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude,Cuisines STR
0,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1437,Jakarta Selatan,DKI Jakarta,-6.239143,106.807954,"GrillHouse,Steak"
1,Pempek Megaria,['Quick Bites'],['Pempek'],['Palembang'],70000,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",4.3,461,Jakarta Pusat,DKI Jakarta,-6.199900,106.843739,Pempek
2,Tjikinii Lima,['Casual Dining'],['Kopi'],"['Indonesia', 'Barat']",300000,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",4.0,391,Jakarta Pusat,DKI Jakarta,-6.189802,106.837150,Kopi
3,Gado-Gado Bon-Bin,['Quick Bites'],['Makanan Sehat'],"['Jawa', 'Indonesia']",90000,"Jl. Cikini 4 No. 5, Cikini, Jakarta",4.0,177,Jakarta Pusat,DKI Jakarta,-6.192959,106.838377,MakananSehat
4,RM Wong Tegal,['Quick Bites'],['Satay'],"['Jawa', 'Indonesia']",100000,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",3.9,12,Jakarta Barat,DKI Jakarta,-6.182315,106.782992,Satay
5,Martabak Elin Bandung,['Dessert Parlor'],"['Martabak', 'Makanan Jalanan']",['Indonesia'],80000,"Jl. Pinang Mas III/A 8, Pondok Indah, Jakarta",3.9,26,Jakarta Selatan,DKI Jakarta,-6.265635,106.778986,"Martabak,MakananJalanan"
6,Bakso Sapi Rudi,['Quick Bites'],"['Bakso', 'Meat']",['Indonesia'],60000,"Jl. Rajawali Selatan I No. 54A, Gunung Sahari,...",3.9,22,Jakarta Pusat,DKI Jakarta,-6.144706,106.837500,"Bakso,Meat"
7,A Siang,['Quick Bites'],['Kopi'],['Chinese'],65000,"Ruko Taman Palem Lestari, Blok B16 No. 9, Jl. ...",3.9,7,Jakarta Barat,DKI Jakarta,-6.139209,106.725797,Kopi
8,Tropicale Juice Bar,['Toko Minuman'],"['Desserts', 'Jus', 'Minuman']",['Indonesia'],50000,"Ampera Garden, Jl. Ampera Raya No. 116, Ciland...",3.9,9,Jakarta Selatan,DKI Jakarta,-6.288935,106.817939,"Desserts,Jus,Minuman"
9,Sate Taichan Bang Tetew,['Quick Bites'],['Satay'],['Indonesia'],80000,"Jl. Danau Sunter Utara, Blok J 12 No. 26-28, S...",3.9,36,Jakarta Utara,DKI Jakarta,-6.137467,106.873776,Satay


In [13]:
cv= CountVectorizer()
cv_csn=cv.fit_transform(rdf_csne['Cuisines STR'])

In [14]:
cvcs_df= pd.DataFrame(cv_csn.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvcs_df.columns

Index(['ayam', 'bakmi', 'bakso', 'barbekyu', 'bebek', 'bubbletea', 'bubur',
       'burger', 'deli', 'desserts', 'dimsum', 'grillhouse', 'gudeg',
       'hanyaminuman', 'icecream', 'jus', 'kebab', 'kopi', 'kue',
       'makanancepatsaji', 'makananjalanan', 'makanansehat', 'martabak',
       'meat', 'minuman', 'pempek', 'pizza', 'ramen', 'roti', 'satay',
       'seafood', 'snacks', 'soto', 'steak', 'sushi', 'tea'],
      dtype='object')

In [15]:
cvcs_df

,ayam,bakmi,bakso,barbekyu,bebek,bubbletea,bubur,burger,deli,desserts,dimsum,grillhouse,gudeg,hanyaminuman,icecream,jus,kebab,kopi,kue,makanancepatsaji,makananjalanan,makanansehat,martabak,meat,minuman,pempek,pizza,ramen,roti,satay,seafood,snacks,soto,steak,sushi,tea
Restaurant Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Holy Smokes,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Pempek Megaria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
Tjikinii Lima,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gado-Gado Bon-Bin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RM Wong Tegal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nock Nock Cafe & Resto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Abun Takoyaki,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
A Wen Seafood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [16]:
cos_sin_csne=cosine_similarity(cv_csn)
pd.DataFrame(cos_sin_csne,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.0   
A Wen Seafood                                        0.0          0.0   
Talaga Kuring                                        0.0          0.0   
Tigalima Coffee & Kitchen                            0.0          0.0   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                  0.000000                     0.0     0.000000   
Pempek Megaria               0.000000                     0.0     0.000000   
Tjikinii Lima                1.000000                     0.0     0.707107   
Gado-Gado Bon-Bin            0.000000                     0.0     0.000000   
RM Wong Tegal                0.000000                     0.0     0.000000   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto       1.000000                     0.0     0.707107   
Abun Takoyaki                0.000000                     0.0     0.000000   
A Wen Seafood                0.000000                     0.0     0.000000   
Talaga Kuring                1.000000                     0.0     0.707107   
Tigalima Coffee & Kitchen    0.707107                     0.0     1.000000   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                          0.0                     0.00000   
Pempek Megaria                       0.0                     0.00000   
Tjikinii Lima                        0.0                     0.00000   
Gado-Gado Bon-Bin                    0.0                     0.00000   
RM Wong Tegal                        0.0                     0.57735   
...                                  ...                         ...   
Nock Nock Cafe & Resto               0.0                     0.00000   
Abun Takoyaki                        0.0                     0.00000   
A Wen Seafood                        0.0                     0.00000   
Talaga Kuring                        0.0                     0.00000   
Tigalima Coffee & Kitchen            0.0                     0.00000   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                                  0.0                  0.000000   
Pempek Megaria                               0.0                  0.000000   
Tjikinii Lima                                0.0                  0.707107   
Gado-Gado Bon-Bin                            0.0                  0.000000   
RM Wong Tegal                                0.0                  0.000000   
...                                          ...                       ...   
Nock Nock Cafe & Resto                       0.0                  0.707107   
Abun Takoyaki                                0.0                  0.000000   
A Wen Seafood                                0.0                  0.000000   
Talaga Kuring                                0.0                  0.707107   
Tigalima Coffee & Kitchen                    0.0                  1.000000   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   0.000000      0.000000   
Tjikinii Lima                                    1.000000      1.000000   
Gado-Gado Bon-Bin                                0.000000      0.000000   
RM Wong Tegal                                    0.000000      0.000000   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           1.000000      1.000000   
Abun Takoyaki                                    0.000000      0.000000   
A Wen Seafood                                    0.00000

In [17]:
def get_recomendation_restaurant(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_csne[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend.loc[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [18]:
get_recomendation_restaurant('Holy Smokes')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1438,Jakarta Selatan,DKI Jakarta,-6.238930,106.807761
2,Poka Ribs,['Casual Dining'],"['Grill House', 'Steak']",['Indonesia'],250000,"Summarecon Mall Serpong, Lantai Ground, Downto...",3.7,96,Tangerang Kabupaten,Banten,-6.241130,106.628486
3,Mamoo,['Quick Bites'],"['Grill House', 'Steak']",['Indonesia'],50000,"Jl. Raden Patah Blok Lebang Baru No.9, Ciledug...",3.4,56,Tangerang Kota,Banten,-6.245393,106.700506
4,Andakar,['Casual Dining'],"['Grill House', 'Steak']",['Indonesia'],250000,"Jl. Cilandak KKO No. 2, Cilandak, Jakarta",3.9,194,Jakarta Selatan,DKI Jakarta,-6.306459,106.814015
5,Tony Roma's,['Fine Dining'],"['Grill House', 'Steak']",['Indonesia'],750000,"UOB Plaza, Lantai Mezzanine, Jl. M. H. Thamrin...",4.6,486,Jakarta Pusat,DKI Jakarta,-6.198943,106.822410


# 1.2 RECOMMENDER - CUISINE REGIONALITY

In [19]:
rdf_csne['Cuisine Regionality STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Cuisine Regionality'), axis=1)

In [20]:
rdf_csne.head()

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude,Cuisines STR,Cuisine Regionality STR
0,Holy Smokes,['Casual Dining'],"['Grill House', 'Steak']",['Barat'],500000,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",4.4,1437,Jakarta Selatan,DKI Jakarta,-6.239143,106.807954,"GrillHouse,Steak",Barat
1,Pempek Megaria,['Quick Bites'],['Pempek'],['Palembang'],70000,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",4.3,461,Jakarta Pusat,DKI Jakarta,-6.199900,106.843739,Pempek,Palembang
2,Tjikinii Lima,['Casual Dining'],['Kopi'],"['Indonesia', 'Barat']",300000,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",4.0,391,Jakarta Pusat,DKI Jakarta,-6.189802,106.837150,Kopi,"Indonesia,Barat"
3,Gado-Gado Bon-Bin,['Quick Bites'],['Makanan Sehat'],"['Jawa', 'Indonesia']",90000,"Jl. Cikini 4 No. 5, Cikini, Jakarta",4.0,177,Jakarta Pusat,DKI Jakarta,-6.192959,106.838377,MakananSehat,"Jawa,Indonesia"
4,RM Wong Tegal,['Quick Bites'],['Satay'],"['Jawa', 'Indonesia']",100000,"Jl. Tanjung Duren Barat 6 No. 5, Tanjung Duren...",3.9,12,Jakarta Barat,DKI Jakarta,-6.182315,106.782992,Satay,"Jawa,Indonesia"


In [21]:
cv= CountVectorizer()
cv_rgn=cv.fit_transform(rdf_csne['Cuisine Regionality STR'])

In [22]:
cvrg_df= pd.DataFrame(cv_rgn.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvrg_df.columns

Index(['aceh', 'amerika', 'amerikalatin', 'arab', 'asia', 'australian', 'bali',
       'bangka', 'barat', 'batak', 'belanda', 'betawi', 'chinese', 'eropa',
       'fusion', 'india', 'indiaselatan', 'indiautara', 'indonesia', 'itali',
       'jawa', 'jepang', 'jerman', 'kalimantan', 'kontinental', 'korea',
       'libanon', 'lombok', 'makassar', 'malaysia', 'manado', 'maroko',
       'medan', 'mediterania', 'meksiko', 'mongolia', 'noregion', 'padang',
       'palembang', 'peranakan', 'perancis', 'peruvian', 'singapura',
       'spanyol', 'sulawesi', 'sumatera', 'sunda', 'taiwanese', 'thai',
       'timurtengah', 'turki', 'vietnam', 'yogyakarta', 'yunani'],
      dtype='object')

In [23]:
cos_sin_rgn=cosine_similarity(cv_rgn)
pd.DataFrame(cos_sin_rgn,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.5     0.707107   
Tigalima Coffee & Kitchen                            0.5     0.707107   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                  0.000000                     0.0     0.000000   
Pempek Megaria               0.000000                     0.0     0.000000   
Tjikinii Lima                0.000000                     0.5     0.000000   
Gado-Gado Bon-Bin            0.000000                     0.5     0.000000   
RM Wong Tegal                0.000000                     0.5     0.000000   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto       0.000000                     0.5     0.707107   
Abun Takoyaki                0.000000                     0.0     0.000000   
A Wen Seafood                0.707107                     0.0     0.000000   
Talaga Kuring                0.000000                     0.5     0.000000   
Tigalima Coffee & Kitchen    0.000000                     0.5     0.707107   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                     1.000000                         0.0   
Pempek Megaria                  0.000000                         0.0   
Tjikinii Lima                   0.707107                         0.5   
Gado-Gado Bon-Bin               0.000000                         1.0   
RM Wong Tegal                   0.000000                         1.0   
...                                  ...                         ...   
Nock Nock Cafe & Resto          0.000000                         0.5   
Abun Takoyaki                   0.000000                         0.0   
A Wen Seafood                   0.000000                         0.0   
Talaga Kuring                   0.000000                         0.5   
Tigalima Coffee & Kitchen       0.000000                         0.5   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                             0.000000                  0.577350   
Pempek Megaria                          0.000000                  0.000000   
Tjikinii Lima                           0.000000                  0.816497   
Gado-Gado Bon-Bin                       0.408248                  0.408248   
RM Wong Tegal                           0.408248                  0.408248   
...                                          ...                       ...   
Nock Nock Cafe & Resto                  0.000000                  0.816497   
Abun Takoyaki                           0.000000                  0.000000   
A Wen Seafood                           0.408248                  0.000000   
Talaga Kuring                           0.000000                  0.408248   
Tigalima Coffee & Kitchen               0.000000                  0.816497   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   0.000000      0.000000   
Tjikinii Lima                                    0.707107      0.707107   
Gado-Gado Bon-Bin                                0.707107      0.707107   
RM Wong Tegal                                    0.707107      0.707107   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           0.707107      0.707107   
Abun Takoyaki                                    0.000000      0.000000   
A Wen Seafood                                    0.000000      0.000000   
Talaga Kuring                                    0.707107      0.707107   
Tigalima Coffee & Kitchen              

In [24]:
def get_recomendation_restaurant(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_rgn[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [25]:
get_recomendation_restaurant('Pempek Megaria')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Pempek Sriwijaya,['Quick Bites'],['Pempek'],['Palembang'],90000,"Jl. Hadiah 1, Blok D14 No. 1204 B, Jelambar, J...",3.5,18,Jakarta Barat,DKI Jakarta,-6.161505,106.783709
2,Pempek Ekspress,['Food Court'],['Pempek'],['Palembang'],70000,"Lippo Mall Puri @ The St. Moritz, Lantai Lower...",2.8,6,Jakarta Barat,DKI Jakarta,-6.188875,106.738983
3,Pempek Alik,['Quick Bites'],['Pempek'],['Palembang'],60000,"Jl. Kedoya Duri No. 24B, Kedoya, Jakarta",3.7,5,Jakarta Barat,DKI Jakarta,-6.181134,106.762256
4,Pempek Cawan Putih,['Quick Bites'],['Pempek'],['Palembang'],80000,"Jl. Pecenongan No. 16, Pecenongan, Jakarta",3.0,10,Jakarta Pusat,DKI Jakarta,-6.166608,106.826504
5,Pempek Emak Seroja,['Quick Bites'],['Pempek'],['Palembang'],100000,"Jl. Ratu Kemuning Blok D2 No. 26C, Green Ville...",3.2,6,Jakarta Barat,DKI Jakarta,-6.169589,106.769300


# 1.3 RECOMMENDER - PLACE TYPE

In [26]:
rdf_csne['Place Type STR'] = rdf_csne.apply(lambda row: minimize_str(row, 'Place Type'), axis=1)

In [27]:
cv= CountVectorizer()
cv_rest=cv.fit_transform(rdf_csne['Place Type STR'])

cvpl_df= pd.DataFrame(cv_rest.todense(),columns=cv.get_feature_names(),index=rdf['Restaurant Name'])
cvpl_df.columns

Index(['bar', 'casualdining', 'dessertparlor', 'finedining', 'foodcourt',
       'kafe', 'kakilima', 'kios', 'lounge', 'quickbites', 'roti', 'tokokue',
       'tokominuman'],
      dtype='object')

In [28]:
cos_sin_rest=cosine_similarity(cv_rest)
pd.DataFrame(cos_sin_rest,index=rdf['Restaurant Name'],columns=rdf['Restaurant Name'])

0.0   
A Wen Seafood                                   1.000000          0.0   
Talaga Kuring                                   1.000000          0.0   
Tigalima Coffee & Kitchen                       0.707107          0.0   

Restaurant Name            Oma Recipe  Soto Betawi Bang Japri  Kopi Ranger  \
Restaurant Name                                                              
Holy Smokes                       0.0                     0.0          0.0   
Pempek Megaria                    0.0                     0.0          0.0   
Tjikinii Lima                     0.0                     0.0          0.0   
Gado-Gado Bon-Bin                 0.0                     0.0          0.0   
RM Wong Tegal                     0.0                     0.0          0.0   
...                               ...                     ...          ...   
Nock Nock Cafe & Resto            0.0                     0.0          0.0   
Abun Takoyaki                     0.0                     0.0          0.0   
A Wen Seafood                     0.0                     0.0          0.0   
Talaga Kuring                     0.0                     0.0          0.0   
Tigalima Coffee & Kitchen         0.0                     0.0          0.0   

Restaurant Name            Quick Chicken  Sate Tegal Kambing Muda 88  \
Restaurant Name                                                        
Holy Smokes                     0.000000                    0.000000   
Pempek Megaria                  1.000000                    1.000000   
Tjikinii Lima                   0.000000                    0.000000   
Gado-Gado Bon-Bin               1.000000                    1.000000   
RM Wong Tegal                   1.000000                    1.000000   
...                                  ...                         ...   
Nock Nock Cafe & Resto          0.707107                    0.707107   
Abun Takoyaki                   1.000000                    1.000000   
A Wen Seafood                   1.000000                    1.000000   
Talaga Kuring                   1.000000                    1.000000   
Tigalima Coffee & Kitchen       0.707107                    0.707107   

Restaurant Name            Rumah Makan Pak Ubung  FIFI Coffee Bar & Bistro  \
Restaurant Name                                                              
Holy Smokes                             0.000000                  0.000000   
Pempek Megaria                          1.000000                  0.000000   
Tjikinii Lima                           0.000000                  0.000000   
Gado-Gado Bon-Bin                       1.000000                  0.000000   
RM Wong Tegal                           1.000000                  0.000000   
...                                          ...                       ...   
Nock Nock Cafe & Resto                  0.707107                  0.707107   
Abun Takoyaki                           1.000000                  0.000000   
A Wen Seafood                           1.000000                  0.000000   
Talaga Kuring                           1.000000                  0.000000   
Tigalima Coffee & Kitchen               0.707107                  0.707107   

Restaurant Name            Nasi Goreng Karee Abang Gandul  Kedai Sutera  \
Restaurant Name                                                           
Holy Smokes                                      0.000000      0.000000   
Pempek Megaria                                   1.000000      1.000000   
Tjikinii Lima                                    0.000000      0.000000   
Gado-Gado Bon-Bin                                1.000000      1.000000   
RM Wong Tegal                                    1.000000      1.000000   
...                                                   ...           ...   
Nock Nock Cafe & Resto                           0.707107      0.707107   
Abun Takoyaki                                    1.000000      1.000000   
A Wen Seafood                                    1.00000

In [29]:
def get_recomendation_restaurant(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_rest[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)
    
    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [30]:
get_recomendation_restaurant('Cirengcrispi')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Coco Tree,"['Food Court', 'Kios']",['Minuman'],['Indonesia'],70000,"Mall of Indonesia, Lantai Lower Ground, Pasar ...",3.1,4,Jakarta Utara,DKI Jakarta,-6.150546,106.891921
2,Soto Ambengan,['Food Court'],['Soto'],"['Jawa', 'Indonesia']",150000,"Mall Kelapa Gading 3, Lantai 3, Eat & Eat, Jl....",3.3,6,Jakarta Utara,DKI Jakarta,-6.158510,106.907039
3,A&W Ice Cream,['Kios'],"['Ice Cream', 'Desserts']",['Indonesia'],50000,"ITC Permata Hijau, Lantai Ground, Jl. Letjen S...",3.4,4,Jakarta Selatan,DKI Jakarta,-6.220934,106.783784
4,Bim Coffee,['Food Court'],"['Kopi', 'Minuman']",['No Region'],100000,"Soekarno Hatta International Airport, Terminal...",2.8,4,Tangerang Kota,Banten,-6.131209,106.653184
5,Singapore Hainanese Chicken Rice,['Food Court'],['Ayam'],['Singapura'],100000,"Senayan City, Lantai 5, Delicaé, Jl. Asia Afri...",3.3,22,Jakarta Pusat,DKI Jakarta,-6.227689,106.797580


# 1.4 RECOMMENDER - ALL

In [31]:
cos_sin_cat=(cos_sin_csne*0.4)+(cos_sin_rgn*0.3)+(cos_sin_rest*0.3)

In [32]:
from math import sin, cos, sqrt, atan2, radians

In [33]:
def get_recomendation_type(title):
    idx=rdf_csne[rdf_csne['Restaurant Name']==title].index[0]
    rest_similar=pd.Series(cos_sin_cat[idx])
    idx_similar=rest_similar.sort_values(ascending=False).head(6).index
    recommend=rdf_csne.loc[idx_similar]
    recommend.reset_index(drop=True,inplace=True)

    return recommend[1:6][['Restaurant Name', 'Place Type', 'Cuisines', 'Cuisine Regionality','Price for 2','Restaurant Address', 'Restaurant Rating','Restaurant Review','City', 'Province', 'Latitude', 'Longitude']]

In [45]:
get_recomendation_type('Cirengcrispi')

,Restaurant Name,Place Type,Cuisines,Cuisine Regionality,Price for 2,Restaurant Address,Restaurant Rating,Restaurant Review,City,Province,Latitude,Longitude
1,Batagor Cuplis,['Food Court'],"['Snacks', 'Makanan Jalanan']","['Sunda', 'Indonesia']",120000,"Food Colony, Jl. HR Rasuna Said, Kuningan, Jak...",2.9,11,Jakarta Selatan,DKI Jakarta,-6.221356,106.833619
2,Jajanan Khas Pontianak @haw,['Food Court'],['Snacks'],['Kalimantan'],50000,"Summarecon Mall Serpong, Lantai 2, Souq Pasar ...",3.5,4,Tangerang Kabupaten,Banten,-6.241025,106.628265
3,Siomay Oenake,['Food Court'],['Snacks'],['Indonesia'],60000,"Transmart Yasmin, Lantai Ground, Bogor Kuliner...",3.6,6,Bogor Kota,Jawa Barat,-6.555770,106.775748
4,Takoyaki Grill House,['Food Court'],['Snacks'],"['Barat', 'Jepang']",50000,"H.O.P (Hang Out Place), Jl. Pluit Putra Raya N...",3.9,5,Jakarta Utara,DKI Jakarta,-6.118368,106.791514
5,Rujak Kolam Medan,['Food Court'],['Snacks'],"['Medan', 'Indonesia']",80000,"Kota Kasablanka, Lantai 2, Eat & Eat, Jl. Casa...",3.3,4,Jakarta Selatan,DKI Jakarta,-6.223185,106.842656


# DEPLOY RECOMMENDER SYSTEM

In [37]:
pickle.dump(cv_csn, open('recom_cuisine.sav', 'wb'))
pickle.dump(cv_rgn, open('recom_cuisine_region.sav', 'wb'))
pickle.dump(cv_rest, open('recom_place.sav', 'wb'))

In [40]:
rdf_csne.drop(['Cuisines STR', 'Cuisine Regionality STR', 'Place Type STR'], axis=1).to_csv("RDF - Recommender System.csv")